In [1]:
import zipfile
import os
import pandas as pd
import deflatebr as dbr
import json
import requests
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import sidrapy

In [2]:
def concatenar_csvs(diretorio):
    frames = []

    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(diretorio, arquivo)
            
            df = pd.read_csv(caminho_arquivo, encoding='utf-8', delimiter=";", decimal=",")
            df = df.rename(columns={df.columns[0]: "data_base"})
            df['data_base'] = pd.to_datetime(df['data_base'], format="%Y-%m-%d")
            df['mes'] = df['data_base'].dt.month #cria uma nova coluna com mês
            df['ano'] = df['data_base'].dt.year
            df = df[df['mes'] == 12]
            df = df[['ano', 'cliente', 'uf', 'ocupacao', 'cnae_secao', 'ativo_problematico']]
            #df.loc[df['ocupacao'] == '-', 'ocupacao'] = df['cnae_secao'] #Adiciona o valor da coluna cnae_secao quando a coluna ocupacao é "-"
            df = df.groupby(['ano','cliente','uf', 'ocupacao', 'cnae_secao'])['ativo_problematico'].sum().reset_index()
            
            frames.append(df)

    df_concatenado = pd.concat(frames, ignore_index=True)

    return df_concatenado

In [3]:
anos = list(range(2012, 2024))
dataframes = []

for ano in anos:
    diretorio = f"planilha_{ano}"
    dataframe_ano = concatenar_csvs(diretorio)
    dataframes.append(dataframe_ano)

df_total = pd.concat(dataframes, ignore_index=False)

In [4]:
populacao_estados = sidrapy.get_table(table_code='4709',
                         territorial_level="3",
                         ibge_territorial_code="all",
                         variable='93') 

In [5]:
populacao_estados = populacao_estados.drop(columns = ["NC", "NN", "MC", "MN","D1C","D2C","D2N","D3C"])

In [6]:
populacao_estados.columns = populacao_estados.iloc[0]

In [7]:
populacao_estados.head(2)

,Valor,Unidade da Federação,Ano
0,Valor,Unidade da Federação,Ano
1,1581016,Rondônia,2022


In [8]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8200 entries, 0 to 759
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ano                 8200 non-null   int32  
 1   cliente             8200 non-null   object 
 2   uf                  8200 non-null   object 
 3   ocupacao            8200 non-null   object 
 4   cnae_secao          8200 non-null   object 
 5   ativo_problematico  8200 non-null   float64
dtypes: float64(1), int32(1), object(4)
memory usage: 416.4+ KB


In [9]:
uf_to_state = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

In [10]:
df_uf_to_state = pd.DataFrame(list(uf_to_state.items()), columns=['uf', 'Estado'])

In [11]:
df_uf_to_state.head(5)

,uf,Estado
0,AC,Acre
1,AL,Alagoas
2,AP,Amapá
3,AM,Amazonas
4,BA,Bahia


In [12]:
df_total = pd.merge(df_total,
                    df_uf_to_state)

In [13]:
df_total.head(5)

,ano,cliente,uf,ocupacao,cnae_secao,ativo_problematico,Estado
0,2012,PF,AC,PF - Aposentado/pensionista,-,11448387.97,Acre
1,2012,PF,AC,PF - Autônomo,-,8955205.34,Acre
2,2012,PF,AC,PF - Empregado de empresa privada,-,6786123.20,Acre
3,2012,PF,AC,PF - Empregado de entidades sem fins lucrativos,-,21911.72,Acre
4,2012,PF,AC,PF - Empresário,-,27433879.37,Acre


In [14]:
populacao_estados.head(5)

,Valor,Unidade da Federação,Ano
0,Valor,Unidade da Federação,Ano
1,1581016,Rondônia,2022
2,830026,Acre,2022
3,3941175,Amazonas,2022
4,636303,Roraima,2022


In [15]:
df_total = pd.merge(df_total,
                    populacao_estados,
                   right_on = "Unidade da Federação",
                   left_on = "Estado",
                   how = "inner")

In [16]:
df_total.head(5)

,ano,cliente,uf,ocupacao,cnae_secao,ativo_problematico,Estado,Valor,Unidade da Federação,Ano
0,2012,PF,AC,PF - Aposentado/pensionista,-,11448387.97,Acre,830026,Acre,2022
1,2012,PF,AC,PF - Autônomo,-,8955205.34,Acre,830026,Acre,2022
2,2012,PF,AC,PF - Empregado de empresa privada,-,6786123.20,Acre,830026,Acre,2022
3,2012,PF,AC,PF - Empregado de entidades sem fins lucrativos,-,21911.72,Acre,830026,Acre,2022
4,2012,PF,AC,PF - Empresário,-,27433879.37,Acre,830026,Acre,2022


In [17]:
df_total['Estado'] = df_total['uf'].map(uf_to_state)
df_total['Estado'] = df_total['Estado'].str.upper() 

In [18]:
df_total['data'] = pd.to_datetime(df_total['ano'], format="%Y")

In [19]:
df_total['ativo_problematico_deflacionado'] = dbr.deflate(nominal_values=df_total['ativo_problematico'], nominal_dates=df_total['data'], real_date='2022-12')

/Users/brunamoura/anaconda3/lib/python3.11/site-packages/deflatebr/deflate.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['deflated'] = df[['nom_values', 'VALVALOR']].apply(lambda x: ((real_indx/x[1]) * x[0])[0], axis=1)


In [20]:
pd.set_option('display.float_format', '{:.2f}'.format)
df_total.head(5)

,ano,cliente,uf,ocupacao,cnae_secao,ativo_problematico,Estado,Valor,Unidade da Federação,Ano,data,ativo_problematico_deflacionado
0,2012,PF,AC,PF - Aposentado/pensionista,-,11448387.97,ACRE,830026,Acre,2022,2012-01-01,21775491.61
1,2012,PF,AC,PF - Autônomo,-,8955205.34,ACRE,830026,Acre,2022,2012-01-01,17033315.02
2,2012,PF,AC,PF - Empregado de empresa privada,-,6786123.20,ACRE,830026,Acre,2022,2012-01-01,12907596.18
3,2012,PF,AC,PF - Empregado de entidades sem fins lucrativos,-,21911.72,ACRE,830026,Acre,2022,2012-01-01,41677.35
4,2012,PF,AC,PF - Empresário,-,27433879.37,ACRE,830026,Acre,2022,2012-01-01,52180814.60


In [21]:
df_ocupacao_pf_ativoproblematico = df_total[df_total['cliente'] == "PF"]

In [22]:
df_ocupacao_pf_ativoproblematico.head(3)

,ano,cliente,uf,ocupacao,cnae_secao,ativo_problematico,Estado,Valor,Unidade da Federação,Ano,data,ativo_problematico_deflacionado
0,2012,PF,AC,PF - Aposentado/pensionista,-,11448387.97,ACRE,830026,Acre,2022,2012-01-01,21775491.61
1,2012,PF,AC,PF - Autônomo,-,8955205.34,ACRE,830026,Acre,2022,2012-01-01,17033315.02
2,2012,PF,AC,PF - Empregado de empresa privada,-,6786123.20,ACRE,830026,Acre,2022,2012-01-01,12907596.18


In [23]:
url = "https://raw.githubusercontent.com/jonates/opendata/master/arquivos_geoespaciais/unidades_da_federacao.json" #Temos que dar os créditos
response = requests.get(url)
geojson_data = response.json()

In [24]:
df_ocupacao_pf_ativoproblematico.columns

Index(['ano', 'cliente', 'uf', 'ocupacao', 'cnae_secao', 'ativo_problematico',
       'Estado', 'Valor', 'Unidade da Federação', 'Ano', 'data',
       'ativo_problematico_deflacionado'],
      dtype='object')

In [26]:
df_ocupacao_pf_ativoproblematico = df_ocupacao_pf_ativoproblematico.drop(columns = ['cliente', 'uf', 'cnae_secao', 'ativo_problematico', 'Ano',
                                                 'data','Unidade da Federação'])

In [27]:
df_ocupacao_pf_ativoproblematico['Valor'] = df_ocupacao_pf_ativoproblematico['Valor'].astype(float)

In [28]:
df_ocupacao_pf_ativoproblematico['ativo_problematico/pop'] = df_ocupacao_pf_ativoproblematico['ativo_problematico_deflacionado'] / df_ocupacao_pf_ativoproblematico['Valor']

In [31]:
df_ocupacao_pf_ativoproblematico['ocupacao']=df_ocupacao_pf_ativoproblematico['ocupacao'].str.replace('PF - ','')

In [32]:
df_ocupacao_pf_ativoproblematico.head(3)

,ano,ocupacao,Estado,Valor,ativo_problematico_deflacionado,ativo_problematico/pop
0,2012,Aposentado/pensionista,ACRE,830026.00,21775491.61,26.23
1,2012,Autônomo,ACRE,830026.00,17033315.02,20.52
2,2012,Empregado de empresa privada,ACRE,830026.00,12907596.18,15.55


In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Análise do Ativo Problemático'),
    
    html.Div([
        html.Label('Selecione uma ocupação:'),
        dcc.Dropdown(
            id='ocupacao-dropdown',
            options=[{'label': i, 'value': i} for i in df_ocupacao_pf_ativoproblematico['ocupacao'].unique()],
            value=df_ocupacao_pf_ativoproblematico['ocupacao'].iloc[0]
        ),
    ]),
    
    dcc.Graph(id='choropleth-map')
])

@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('ocupacao-dropdown', 'value')]
)
def update_choropleth(ocupacao_value):
    
    filtered_df = df_ocupacao_pf_ativoproblematico[df_ocupacao_pf_ativoproblematico['ocupacao'] == ocupacao_value]
    
    fig = px.choropleth_mapbox(filtered_df, 
                               geojson=geojson_data, 
                               locations='Estado', 
                               color='ativo_problematico/pop',
                               color_continuous_scale="sunsetdark",
                               range_color=(0, max(filtered_df['ativo_problematico/pop'])),
                               animation_frame='ano', 
                               mapbox_style="open-street-map",
                               zoom=3, 
                               center={"lat": -17.14, "lon": -57.33},
                               opacity=1,
                               labels={'ativo_problematico/pop':'Carteira Ativa',
                                       'uf': 'Unidade da Federação do Brasil'},
                               featureidkey="properties.NM_ESTADO")
    
    fig.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [33]:
df_ocupacao_pf_ativoproblematico.to_csv("df_ocupacao_pf_ativoproblematico.csv")

In [34]:
df_cnae_pj_ativoproblematico = df_total[df_total['cliente'] == "PJ"]

In [35]:
df_cnae_pj_ativoproblematico.head(3)

,ano,cliente,uf,ocupacao,cnae_secao,ativo_problematico,Estado,Valor,Unidade da Federação,Ano,data,ativo_problematico_deflacionado
7,2012,PJ,AC,-,"PJ - Administração pública, defesa e seguridad...",287.33,ACRE,830026,Acre,2022,2012-01-01,546.52
8,2012,PJ,AC,-,"PJ - Agricultura, pecuária, produção florestal...",635844.18,ACRE,830026,Acre,2022,2012-01-01,1209412.16
9,2012,PJ,AC,-,PJ - Alojamento e alimentação,3776846.30,ACRE,830026,Acre,2022,2012-01-01,7183778.64


In [36]:
df_cnae_pj_ativoproblematico.columns

Index(['ano', 'cliente', 'uf', 'ocupacao', 'cnae_secao', 'ativo_problematico',
       'Estado', 'Valor', 'Unidade da Federação', 'Ano', 'data',
       'ativo_problematico_deflacionado'],
      dtype='object')

In [37]:
df_cnae_pj_ativoproblematico = df_cnae_pj_ativoproblematico.drop(columns = ['cliente', 'uf', 'ocupacao', 'ativo_problematico', 'Unidade da Federação', 'Ano',
                                                 'data'])

In [38]:
df_cnae_pj_ativoproblematico.head(5)

,ano,cnae_secao,Estado,Valor,ativo_problematico_deflacionado
7,2012,"PJ - Administração pública, defesa e seguridad...",ACRE,830026,546.52
8,2012,"PJ - Agricultura, pecuária, produção florestal...",ACRE,830026,1209412.16
9,2012,PJ - Alojamento e alimentação,ACRE,830026,7183778.64
10,2012,"PJ - Artes, cultura, esporte e recreação",ACRE,830026,890417.99
11,2012,PJ - Atividades administrativas e serviços com...,ACRE,830026,3991531.14


In [39]:
df_cnae_pj_ativoproblematico['Valor'] = df_cnae_pj_ativoproblematico['Valor'].astype(float)

In [40]:
df_cnae_pj_ativoproblematico['ativo_problematico/pop'] = df_cnae_pj_ativoproblematico['ativo_problematico_deflacionado'] / df_cnae_pj_ativoproblematico['Valor']

In [43]:
df_cnae_pj_ativoproblematico['cnae_secao']=df_cnae_pj_ativoproblematico['cnae_secao'].str.replace('PJ - ','')

In [44]:
df_cnae_pj_ativoproblematico.head(5)

,ano,cnae_secao,Estado,Valor,ativo_problematico_deflacionado,ativo_problematico/pop
7,2012,"Administração pública, defesa e seguridade social",ACRE,830026.00,546.52,0.00
8,2012,"Agricultura, pecuária, produção florestal, pes...",ACRE,830026.00,1209412.16,1.46
9,2012,Alojamento e alimentação,ACRE,830026.00,7183778.64,8.65
10,2012,"Artes, cultura, esporte e recreação",ACRE,830026.00,890417.99,1.07
11,2012,Atividades administrativas e serviços compleme...,ACRE,830026.00,3991531.14,4.81


In [45]:
df_cnae_pj_ativoproblematico.to_csv("df_cnae_pj_ativoproblematico.csv")

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Análise do Ativo Problemático'),
    
    html.Div([
        html.Label('Selecione um setor de atuação:'),
        dcc.Dropdown(
            id='cnae-dropdown',
            options=[{'label': i, 'value': i} for i in df_cnae_pj_ativoproblematico['cnae_secao'].unique()],
            value=df_cnae_pj_ativoproblematico['cnae_secao'].iloc[0]
        ),
    ]),
    
    dcc.Graph(id='choropleth-map')
])

@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('cnae-dropdown', 'value')]
)
def update_choropleth(cnae_value):
    
    filtered_df = df_cnae_pj_ativoproblematico[df_cnae_pj_ativoproblematico['cnae_secao'] == cnae_value]
    
    fig = px.choropleth_mapbox(filtered_df, 
                               geojson=geojson_data, 
                               locations='Estado', 
                               color='ativo_problematico/pop',
                               color_continuous_scale="sunsetdark",
                               range_color=(0, max(filtered_df['ativo_problematico/pop'])),
                               animation_frame='ano', 
                               mapbox_style="open-street-map",
                               zoom=3, 
                               center={"lat": -17.14, "lon": -57.33},
                               opacity=1,
                               labels={'ativo_problematico/pop':'Carteira Ativa',
                                       'uf': 'Unidade da Federação do Brasil'},
                               featureidkey="properties.NM_ESTADO")
    
    fig.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)